In [1]:
import numpy as np

import datamanager
import torch
import pathlib

config = datamanager.HyFluidNeRFDataManagerConfig(dataparser=datamanager.HyFluidDataParserConfig())
config.dataparser.data = pathlib.Path("../data/ScalarReal")
manager: datamanager.HyFluidNeRFDataManager = config.setup(device=torch.device("cuda"))

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


Caching all 480 images.

Output()

Caching all 120 images.

Output()

In [2]:
ray_bundle, batch = manager.next_train()
print(ray_bundle.shape)
print(batch['image'].shape)
print(batch['indices'].shape)

torch.Size([1024])
torch.Size([1024, 3])
torch.Size([1024, 3])


In [3]:
from nerfstudio.model_components.ray_samplers import UniformSampler
from nerfstudio.model_components.scene_colliders import NearFarCollider
from nerfstudio.cameras.rays import RaySamples

collider = NearFarCollider(near_plane=1.1, far_plane=1.5)
sampler_uniform = UniformSampler(num_samples=192)

collider(ray_bundle)
ray_samples_uniform: RaySamples = sampler_uniform(ray_bundle)

print(ray_samples_uniform.shape)

torch.Size([1024, 192])


In [4]:
start_positions = ray_samples_uniform.frustums.get_start_positions()
positions = ray_samples_uniform.frustums.get_positions()

print(start_positions.shape)
print(positions.shape)

torch.Size([1024, 192, 3])
torch.Size([1024, 192, 3])


In [5]:
np.savez_compressed(
    "positions.npz",
    start_positions=start_positions.cpu().numpy(),
    positions=positions.cpu().numpy()
)

In [6]:
image_indices = batch['indices'][:, 0]
all_frames = manager.train_dataset.metadata['all_frames']
frames = datamanager.image_idx_to_frame(image_indices, all_frames)
frames_expanded = frames.to(positions.device).view(positions.shape[0], 1, 1).expand(-1, positions.shape[1], -1)
xyzt = torch.cat((positions, frames_expanded), dim=-1)

print(all_frames)
print(image_indices)
print(frames)
print(frames.shape)
print(xyzt.shape)

[120, 120, 120, 120]
tensor([167.9595, 104.3906, 225.7043,  ..., 239.0000, 210.3843,   0.8230])
tensor([ 47.9595, 104.3906, 105.7043,  ..., 119.0000,  90.3843,   0.8230])
torch.Size([1024])
torch.Size([1024, 192, 4])


In [8]:
import encoder
import taichi as ti

ti.init(arch=ti.cuda)
xyzt_encoder = encoder.HashEncoderHyFluid(
    min_res=np.array([16, 16, 16, 16]),
    max_res=np.array([256, 256, 256, 128]),
    num_scales=16,
    max_params=2 ** 19,
)

xyzt_flat = xyzt.reshape(-1, 4)
xyzt_encoded = xyzt_encoder(xyzt)

[Taichi] Starting on arch=cuda


RuntimeError: [taichi/rhi/cuda/cuda_driver.h:taichi::lang::CUDADriverFunction<void * *,unsigned __int64,void *>::operator ()@92] CUDA Error CUDA_ERROR_OUT_OF_MEMORY: out of memory while calling malloc_async_impl (cuMemAllocAsync)